In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn

In [4]:
# Load Data
train_df = pd.read_csv("../data/gold_recovery_train_new.csv")
test_df = pd.read_csv("../data/gold_recovery_test_new.csv")
full_df = pd.read_csv("../data/gold_recovery_full_new.csv")

# Load Data
train_df = pd.read_csv("../data/gold_recovery_train_new.csv")
test_df = pd.read_csv("../data/gold_recovery_test_new.csv")
full_df = pd.read_csv("../data/gold_recovery_full_new.csv")

In [ ]:
#